In [1]:
from net.st_gcn import Model
import torch


In [9]:
# Cấu hình graph
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

graph_args = {'layout': 'emotion_walk', 'strategy': 'uniform'}

model = Model(
    in_channels=3,          # XYZ
    num_class= 4 ,
    graph_args=graph_args,
    edge_importance_weighting=True
).to(device)

In [10]:
model

Model(
  (data_bn): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (st_gcn_networks): ModuleList(
    (0): st_gcn(
      (gcn): ConvTemporalGraphical(
        (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (tcn): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(9, 1), stride=(1, 1), padding=(4, 0))
        (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): Dropout(p=0, inplace=True)
      )
      (relu): ReLU(inplace=True)
    )
    (1-3): 3 x st_gcn(
      (gcn): ConvTemporalGraphical(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      )
      (tcn): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(9, 1), s

In [11]:
import get_data
data_path = "data/Data.h5"
label_path = "data/Labels_My.h5"  # chỉnh đúng đường dẫn
Gait = get_data.get_data(data_path)
Labels = get_data.get_labels(label_path)

Số dataset đã lấy: 672
Kích thước mảng cuối cùng: (672, 30, 48)
Tổng giá trị hàng 1 của tất cả dataset gộp lại:
(672,)


In [27]:
X = Gait
y = Labels

In [28]:
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split

# Giả sử X là numpy array (672, 30, 48)
N, T, total_features = X.shape
V, C = 16, 3

# Reshape sang (N, T, V, C)
X_reshaped = X.reshape(N, T, V, C)

# Split dữ liệu numpy
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(
    X_reshaped, y, test_size=0.2, random_state=42, shuffle=True)

# Chuyển sang tensor và permute + thêm chiều M
def preprocess_input(x_np):
    x_tensor = torch.tensor(x_np, dtype=torch.float32)
    x_perm = x_tensor.permute(0, 3, 1, 2)  # [N, C, T, V]
    x_final = x_perm.unsqueeze(-1)          # [N, C, T, V, M=1]
    return x_final

X_train_tensor = preprocess_input(X_train_np)
X_test_tensor = preprocess_input(X_test_np)

y_train_tensor = torch.tensor(y_train_np, dtype=torch.long)
y_test_tensor = torch.tensor(y_test_np, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)



In [32]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(200):  # ví dụ 20 epochs
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(y_batch).sum().item()
        total += y_batch.size(0)

    acc = 100. * correct / total
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Accuracy: {acc:.2f}%")


Epoch 1, Loss: 21.4266, Accuracy: 75.42%
Epoch 2, Loss: 14.8099, Accuracy: 83.99%
Epoch 3, Loss: 14.4835, Accuracy: 83.24%
Epoch 4, Loss: 11.8976, Accuracy: 89.20%
Epoch 5, Loss: 13.9170, Accuracy: 84.36%
Epoch 6, Loss: 11.7279, Accuracy: 87.52%
Epoch 7, Loss: 10.4141, Accuracy: 90.13%
Epoch 8, Loss: 9.3884, Accuracy: 89.76%
Epoch 9, Loss: 8.9480, Accuracy: 90.50%
Epoch 10, Loss: 11.7217, Accuracy: 88.08%
Epoch 11, Loss: 9.8430, Accuracy: 89.20%
Epoch 12, Loss: 8.2304, Accuracy: 91.43%
Epoch 13, Loss: 6.6731, Accuracy: 92.55%
Epoch 14, Loss: 9.0016, Accuracy: 90.69%
Epoch 15, Loss: 9.5336, Accuracy: 92.36%
Epoch 16, Loss: 7.8840, Accuracy: 91.81%
Epoch 17, Loss: 10.1773, Accuracy: 88.08%
Epoch 18, Loss: 9.9905, Accuracy: 90.32%
Epoch 19, Loss: 8.0206, Accuracy: 91.99%
Epoch 20, Loss: 6.0859, Accuracy: 93.30%
Epoch 21, Loss: 3.2410, Accuracy: 97.39%
Epoch 22, Loss: 5.2670, Accuracy: 95.34%
Epoch 23, Loss: 5.1461, Accuracy: 94.79%
Epoch 24, Loss: 4.8144, Accuracy: 95.53%
Epoch 25, Loss: 

In [33]:
model.eval()  # Chuyển model sang chế độ đánh giá (eval)
correct = 0
total = 0
with torch.no_grad():  # Không tính gradient khi test, tiết kiệm bộ nhớ
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        outputs = model(X_batch)
        _, predicted = outputs.max(1)
        correct += predicted.eq(y_batch).sum().item()
        total += y_batch.size(0)

test_acc = 100. * correct / total
print(f"Test Accuracy: {test_acc:.2f}%")


Test Accuracy: 94.81%


In [31]:
test_loader